In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import matplotlib.cm as cm
import cartopy.crs as ccrs
import cartopy as cy

In [2]:
class TaiESM():
    def __init__(self, data):
        self.data = xr.open_dataset(data)
        
        self.LON = self.getVar("lon")
        self.LAT = self.getVar("lat")
        self.LON2, self.LAT2 = np.meshgrid(self.LON, self.LAT)
        self.LEV_PRE = self.getLevPre()
    
    def getVar(self, var):
        if type(var) == str:
            return self.data[var].to_numpy()
        elif type(var) == list or type(var) == np.ndarray:
            retVal = []
            for i in range(len(var)):
                retVal.append(self.data[var[i]].to_numpy())
            return retVal
        
    def getLevPre(self):
        PS_tile = np.tile(self.getVar("PS"), (30, 1, 1))
        hybm_tile = np.tile(self.getVar("hybm"), (192, 288, 1)).swapaxes(0, 2).swapaxes(1, 2)
        PRS_lev = np.tile(self.getVar("P0")*self.getVar("hybm"), (192, 288, 1)).swapaxes(0, 2).swapaxes(1, 2) + PS_tile * hybm_tile
        return PRS_lev

In [13]:
left_T, right_T, split_T = 240, 300, 11
left_LHF, right_LHF, split_LHF = -60, 240, 11

In [16]:
for i in range(1, 13):
    tmp = str(i).zfill(2)
    ds_ctl = TaiESM(f"/work/aaron900129/taiesm_work/archive/f09.F2000.Aaron_10years/atm/hist/f09.F2000.Aaron_10years.cam.h0.2000-{tmp}.nc")
    ds_dessert = TaiESM(f"/work/aaron900129/taiesm_work/archive/f09.F2000.Aaron_desert/atm/hist/f09.F2000.Aaron_desert.cam.h0.2000-{tmp}.nc")

    TS_ctl = np.asarray(ds_ctl.getVar("TS"))[0]
    LHFLX_ctl = np.asarray(ds_ctl.getVar("LHFLX"))[0]

    TS_dessert = np.asarray(ds_dessert.getVar("TS"))[0]
    LHFLX_dessert = np.asarray(ds_dessert.getVar("LHFLX"))[0]

    fig, ax = plt.subplots(2, 3, figsize=(15, 8), subplot_kw={'projection': ccrs.PlateCarree()}, dpi=300)
    cs0 = ax[0, 0].contourf(ds_ctl.LON2, ds_ctl.LAT2, TS_ctl, extend='both', levels=np.linspace(left_T, right_T, split_T))
    cs1 = ax[0, 1].contourf(ds_dessert.LON2, ds_dessert.LAT2, TS_dessert, extend='both', levels=np.linspace(left_T, right_T, split_T))
    fig.colorbar(cs0, ax=ax[0, :2], orientation='horizontal', ticks=np.linspace(left_T, right_T, split_T), aspect=30)

    cs2 = ax[0, 2].contourf(ds_ctl.LON2, ds_ctl.LAT2, TS_dessert - TS_ctl, extend='both', levels=np.linspace(-15, 15, 7))
    fig.colorbar(cs2, ax=ax[0, 2:], orientation='horizontal', aspect=15, ticks=np.linspace(-15, 15, 7))


    ax[0, 0].set_title("Control Run Surface Temperature")
    ax[0, 1].set_title("Bare Soil Run Surface Temperature")
    ax[0, 2].set_title("Surface Temperature Difference (Bare Soil - Control)")

    cs3 = ax[1, 0].contourf(ds_ctl.LON2, ds_ctl.LAT2, LHFLX_ctl, extend='both', levels=np.linspace(left_LHF, right_LHF, split_LHF))
    cs4 = ax[1, 1].contourf(ds_dessert.LON2, ds_dessert.LAT2, LHFLX_dessert, extend='both', levels=np.linspace(left_LHF, right_LHF, split_LHF))
    fig.colorbar(cs3, ax=ax[1, :2], orientation='horizontal', ticks=np.linspace(left_LHF, right_LHF, split_LHF), aspect=30)

    cs5 = ax[1, 2].contourf(ds_dessert.LON2, ds_dessert.LAT2, LHFLX_dessert - LHFLX_ctl, extend='both', levels=np.linspace(-60, 240, 7))
    fig.colorbar(cs5, ax=ax[1, 2:], orientation='horizontal', aspect=15, ticks=np.linspace(-60, 240, 7))


    ax[1, 0].set_title("Control Run Latent Heat Flux")
    ax[1, 1].set_title("Bare Soil Run Latent Heat Flux")
    ax[1, 2].set_title("Latent Heat Flux Difference (Bare Soil - Control)")

    for f in range(2):
        for g in range(3):
            gl = ax[f, g].gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth = 0.5)
            gl.xlabels_top = False
            gl.ylabels_right = False
            ax[f, g].add_feature(cy.feature.BORDERS)
            ax[f, g].coastlines()
    plt.savefig(f"../graphs/{i}.png", dpi=300)
    plt.close()
    # plt.show()

/work/aaron900129/miniconda/envs/Aaron/lib/python3.8/site-packages/cartopy/mpl/gridliner.py:451: UserWarning: The .xlabels_top attribute is deprecated. Please use .top_labels to toggle visibility instead.
  warnings.warn('The .xlabels_top attribute is deprecated. Please '
/work/aaron900129/miniconda/envs/Aaron/lib/python3.8/site-packages/cartopy/mpl/gridliner.py:487: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


In [3]:
TS_ctl_all, LHFLX_ctl_all = [], []
TS_dessert_all, LHFLX_dessert_all = [], []
for i in range(1, 13):
    tmp = str(i).zfill(2)
    ds_ctl = TaiESM(f"/work/aaron900129/taiesm_work/archive/f09.F2000.Aaron_10years/atm/hist/f09.F2000.Aaron_10years.cam.h0.2000-{tmp}.nc")
    ds_dessert = TaiESM(f"/work/aaron900129/taiesm_work/archive/f09.F2000.Aaron_desert/atm/hist/f09.F2000.Aaron_desert.cam.h0.2000-{tmp}.nc")

    TS_ctl = np.asarray(ds_ctl.getVar("TS"))[0]
    LHFLX_ctl = np.asarray(ds_ctl.getVar("LHFLX"))[0]

    TS_dessert = np.asarray(ds_dessert.getVar("TS"))[0]
    LHFLX_dessert = np.asarray(ds_dessert.getVar("LHFLX"))[0]

    TS_ctl_all.append(TS_ctl)
    LHFLX_ctl_all.append(LHFLX_ctl)

    TS_dessert_all.append(TS_dessert)
    LHFLX_dessert_all.append(LHFLX_dessert)

TS_ctl_all = np.nanmean(np.asarray(TS_ctl_all), axis=0)
LHFLX_ctl_all = np.nanmean(np.asarray(LHFLX_ctl_all), axis=0)
TS_dessert_all = np.nanmean(np.asarray(TS_dessert_all), axis=0)
LHFLX_dessert_all = np.nanmean(np.asarray(LHFLX_dessert_all), axis=0)

In [7]:
left_T, right_T, split_T = 240, 300, 11
left_LHF, right_LHF, split_LHF = -60, 240, 11

In [12]:
fig, ax = plt.subplots(2, 3, figsize=(17, 8), subplot_kw={'projection': ccrs.PlateCarree()})
cs0 = ax[0, 0].contourf(ds_ctl.LON2, ds_ctl.LAT2, TS_ctl_all, extend='both', levels=np.linspace(left_T, right_T, split_T))
cs1 = ax[0, 1].contourf(ds_dessert.LON2, ds_dessert.LAT2, TS_dessert_all, extend='both', levels=np.linspace(left_T, right_T, split_T))
fig.colorbar(cs0, ax=ax[0, :2], orientation='horizontal', ticks=np.linspace(left_T, right_T, split_T), aspect=30)

cs2 = ax[0, 2].contourf(ds_ctl.LON2, ds_ctl.LAT2, TS_dessert_all - TS_ctl_all, extend='both', levels=np.linspace(-15, 15, 7))
fig.colorbar(cs2, ax=ax[0, 2:], orientation='horizontal', aspect=15, ticks=np.linspace(-15, 15, 7))


ax[0, 0].set_title("Control Run Mean Surface Temperature")
ax[0, 1].set_title("Bare Soil Run Mean Surface Temperature")
ax[0, 2].set_title("Mean Surface Temperature Difference (Bare Soil - Control)")

cs3 = ax[1, 0].contourf(ds_ctl.LON2, ds_ctl.LAT2, LHFLX_ctl_all, extend='both', levels=np.linspace(left_LHF, right_LHF, split_LHF))
cs4 = ax[1, 1].contourf(ds_dessert.LON2, ds_dessert.LAT2, LHFLX_dessert_all, extend='both', levels=np.linspace(left_LHF, right_LHF, split_LHF))
fig.colorbar(cs3, ax=ax[1, :2], orientation='horizontal', ticks=np.linspace(left_LHF, right_LHF, split_LHF), aspect=30)

cs5 = ax[1, 2].contourf(ds_dessert.LON2, ds_dessert.LAT2, LHFLX_dessert_all - LHFLX_ctl_all, extend='both', levels=np.linspace(-60, 60, 7))
fig.colorbar(cs5, ax=ax[1, 2:], orientation='horizontal', aspect=15, ticks=np.linspace(-60, 60, 7))


ax[1, 0].set_title("Control Run Mean Latent Heat Flux")
ax[1, 1].set_title("Bare Soil Run Mean Latent Heat Flux")
ax[1, 2].set_title("Mean Latent Heat Flux Difference (Bare Soil - Control)")

for f in range(2):
    for g in range(3):
        gl = ax[f, g].gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth = 0.5)
        gl.xlabels_top = False
        gl.ylabels_right = False
        ax[f, g].add_feature(cy.feature.BORDERS)
        ax[f, g].coastlines()
plt.savefig(f"../graphs/all_mean.png", dpi=300)
# plt.show()
plt.close()